In [1]:
source("sources/ggrare.R") #github library: https://rdrr.io/github/gauravsk/ranacapa/
source("sources/miseqR.R")
source("sources/qiime2R.R") #github https://github.com/jbisanz/qiime2R
source('sources/abundances.R') #github https://github.com/microbiome/microbiome

library(biomformat)
library(ape)
library(cowplot)
library(data.table)
library(dplyr)
library("expss")
library(ggplot2)
library(phyloseq)
library(scales)
library(tibble)
library(vegan)
library(yaml)
library(ggpubr)
library(cluster)
library(Hmisc)

Warning message:
"package 'ape' was built under R version 3.6.3"
Warning message:
"package 'cowplot' was built under R version 3.6.3"

********************************************************

Note: As of version 1.0.0, cowplot does not change the

  default ggplot2 theme anymore. To recover the previous

  behavior, execute:
  theme_set(theme_cowplot())

********************************************************


Warning message:
"package 'data.table' was built under R version 3.6.3"
Warning message:
"package 'dplyr' was built under R version 3.6.3"

Attaching package: 'dplyr'


The following objects are masked from 'package:data.table':

    between, first, last


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Warning message:
"package 'expss' was built under R version 3.6.3"

Attaching package: 'expss'


The following objects are masked from 'package:dplyr':

  

In [2]:
proj_dir='/Users/slsevilla/Google Drive/MyDocuments_Current/Education/George Mason University/Dissertation/Data/Aim1/'
out_dir=paste(proj_dir,'output/',sep="")
img_dir=paste(proj_dir,out_dir,'img/',sep="")

In [3]:
#Create file of all benchmarking files
benchmark_list = list.files(paste(out_dir,'taxonomic_classification',sep=""),pattern="(?:gg|SILVA|silva).qza")
write.csv(benchmark_list,paste(out_dir,'taxonomic_classification/summary.csv',sep=""))

In [4]:
#decide which files to include, add Y or N flag
benchmarkdf = read.csv(paste(out_dir,'taxonomic_classification/summary_in.csv',sep=""))
rownames(benchmarkdf) = benchmarkdf$x
benchmarkdf = benchmarkdf[c(-1)]
benchmarkdf = subset(benchmarkdf,include=="Y")
benchmarkdf

,x,group.type,group.ref,tax.type,tax.ref,include
,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
blast_cl_SILVA_silva.qza,blast_cl_SILVA_silva.qza,cl,silva,blast,silva,Y
blast_cl_gg_gg.qza,blast_cl_gg_gg.qza,cl,gg,blast,gg,Y
blast_dn_gg.qza,blast_dn_gg.qza,dn,,blast,gg,Y
blast_dn_silva.qza,blast_dn_silva.qza,dn,,blast,silva,Y
blast_op_SILVA_silva.qza,blast_op_SILVA_silva.qza,op,silva,blast,silva,Y
blast_op_gg_gg.qza,blast_op_gg_gg.qza,op,gg,blast,gg,Y
scikit_clSILVA_SILVA.qza,scikit_clSILVA_SILVA.qza,cl,silva,scikit,silva,Y
scikit_clgg_gg.qza,scikit_clgg_gg.qza,cl,gg,scikit,gg,Y
scikit_dada_SILVA.qza,scikit_dada_SILVA.qza,dada,,scikit,silva,Y


In [5]:
#Testing purposes
benchmarkdf = subset(benchmarkdf,x=="scikit_dada_SILVA.qza" | x=="scikit_dada_gg.qza") #x is filename
benchmarkdf

,x,group.type,group.ref,tax.type,tax.ref,include
,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
scikit_dada_SILVA.qza,scikit_dada_SILVA.qza,dada,,scikit,silva,Y
scikit_dada_gg.qza,scikit_dada_gg.qza,dada,,scikit,gg,Y


In [6]:
#Read in taxonomy file
CONTdf = read.csv('../taxonomy/Taxonomy_Complete.csv',stringsAsFactors = FALSE)

#Convert the tax file to SILVA naming with "D_5__"
CONTdf_s = CONTdf
for (i in 1:nrow(CONTdf)){
    old = as.character(CONTdf[i,"Genus"])
    new = paste("D_5__",old,sep="")
    new = gsub(" ", ".", new, fixed = TRUE)
    CONTdf_s[i,"Genus"] = new
    
    old = as.character(CONTdf[i,"Family"])
    new = paste("D_4__",old,sep="")
    new = gsub(" ", ".", new, fixed = TRUE)
    CONTdf_s[i,"Family"] = new
}

#Convert the tax file to gg naming with "f"
CONTdf_g = CONTdf
for (i in 1:nrow(CONTdf)){
    old = as.character(CONTdf[i,"Genus"])
    new = paste("g__",old,sep="")
    new = gsub(" ", ".", new, fixed = TRUE)
    CONTdf_g[i,"Genus"] = new
    
    old = as.character(CONTdf[i,"Family"])
    new = paste("f__",old,sep="")
    new = gsub(" ", ".", new, fixed = TRUE)
    CONTdf_g[i,"Family"] = new
}

In [7]:
phylo_create <- function(tax_in){
    #Read OTUS
    otus<-read_qza('../output/denoising/feature_tables/merged_filtered.qza')

    #Read Greengenes taxonomy file
    taxonomy<-read_qza(tax_in)

    #Create taxonomy table
    tax_table = data.frame()
    col = 1
    row = 1

    for (i in taxonomy$data$Taxon){
        tax_list = as.list(strsplit(as.character(i),";"))
        for (i in 1:lengths(tax_list)){
            tax_table[row,col] = tax_list[[1]][i]
            col = col + 1
        }
        col = 1
        row = row + 1
    }
    rownames(tax_table)<-taxonomy$data$Feature.ID
    colnames(tax_table)<-c("Domain", "Phylum", "Class", "Order", "Family", "Genus", "Species")
    tax_convert<-as.matrix(tax_table)  

    #read metadata
    metadata<-read.table('../manifest/m_cleaned.txt',sep='\t', header=T, row.names=1, comment="")

    #Create phylo object
    phy_obj<-phyloseq(otu_table(otus$data, taxa_are_rows = T), tax_table(tax_convert), sample_data(metadata))
    print(phy_obj) 
    return(phy_obj)
}   

In [8]:
phy_seq <- function(phy_in){
    #glom to tax level, subset for only seq controls
    seq_obj = tax_glom(phy_in,taxrank="Genus") #only merges genera that have the same upper taxonomy
    seq_obj = subset_samples(seq_obj,Sample.Type=="Seq_Control" & Sample.Des!="Seq_Blank")
    print(paste("number of taxa before:", ntaxa(phy_in)))
    print(paste("number of taxa after glom: ",ntaxa(seq_obj)))
    return(seq_obj)
}

In [9]:
otu_extract <- function(phy_in){
    # Extract abundance matrix from the phyloseq object
    OTU1 = as(otu_table(phy_in), "matrix")
    if(taxa_are_rows(phy_in)){OTU1 <- t(OTU1)} # transpose
    OTUdf = as.data.frame(OTU1)# Coerce to data.frame
    return(OTUdf)
}

In [10]:
tax_extract <- function(phy_in){
    #extract taxonomy information into dataframe
    TAXdf = tax_table(phy_in)
    return(TAXdf)
}

In [11]:
otu_tax_clean <- function(OTU_in,TAX_in){
    #Remove zero cols
    OTUdf_clean = OTU_in[, which(colSums(OTU_in) != 0)]

    #rename col from features to genus
    for (i in colnames(OTU_in)){
        newname = paste(as.character(TAX_in[i,"Family"]),as.character(TAX_in[i,"Genus"]),sep="")
        
        #standardize gg naming
        newname = trimws(newname) #remove first space in gg naming
        newname = sub(" g__","g__",newname)
        newname = sub("\\[","",newname)
        newname = sub("\\]","",newname)
        
        if(is.na(newname)){
            next;
        } else{
            names(OTUdf_clean)[names(OTUdf_clean) == i] <- newname
        }
    }
    
    #add metadata col
    metadata<-read.table('../manifest/m_cleaned.txt',sep='\t', header=T, row.names=1, comment="")
    for (a in rownames(OTUdf_clean)){
        OTUdf_clean[a,"Subject.ID"] = metadata[a,"Subject.ID"]
    }
    OTUdf_clean = data.frame(OTUdf_clean,stringsAsFactors=FALSE)
    
    return(OTUdf_clean)
}

In [12]:
#Create TP,FP,FN dataframe
bench_compare <- function(OTUdf_in,CONTdf_in){
    
    F1df = OTUdf_in
    F1df$Sample_names = rownames(F1df) #create col of sample_names
    
    #For each of the controls in the cleaned df
    for (cont in unique(OTUdf_in$Subject.ID)){
        temp=filter(CONTdf_in, get(cont) !=0) #create df of taxa related to control
        
        #Create taxa list, merging family and genus - this will help avoid id errors from taxonomic differences
        #IE one taxa labeled at the genus level may be from family1 and the same taxa may also be from family2
        taxa_list=list()
        for (x in rownames(temp)){taxa_list = c(taxa_list,paste(temp[x,"Family"],temp[x,"Genus"],sep=""))}  
        
        #Create list of samples that match the control
        sample_list = filter(F1df, Subject.ID == cont)$Sample_names
        
        #If the data is not in the df (FN), then add the taxa as a new col
        for (taxa in taxa_list){ 
            if(!(taxa %in% unique(colnames(F1df)))){
                F1df[taxa[!(taxa %in% colnames(F1df))]] = 0
            } 
        }

        for (sample in sample_list){
            for (col in unique(colnames(F1df))[unique(colnames(F1df)) != "Subject.ID" & unique(colnames(F1df)) != "Sample_names"]){ 
                #removes subjectID, sample_names cols from loop
                value_check = F1df[sample,col]

                if(col %in% taxa_list && value_check > 0){ #column matches expected taxa, has features
                    F1df[sample,col] = "TP"
                } else if (col %in% taxa_list && value_check == 0){ #column matches expected taxa, no features
                    F1df[sample,col] = "FN"
                } else if ((!col %in% taxa_list) && value_check > 0){ #column doesnt match expected taxa, has features
                    F1df[sample,col] = "FP"
                } else{ #column doesnt match expected taxa, has no features
                    next
                }
            }
        }
    }
    return(F1df)
}

In [13]:
#Create counts of FP, FN, TP
bench_counts <- function(OTUdf_in,F1_in,bmark_in){
    F1df_counts = subset(OTUdf_in, select=c("Subject.ID"))

    for (sample in rownames(F1df)){
        FN = rowSums(F1_in=="FN")[sample][[1]]
        FP = rowSums(F1_in=="FP")[sample][[1]]
        TP = rowSums(F1_in=="TP")[sample][[1]]

        F1df_counts[sample,"check"]= FN + TP
        F1df_counts[sample,"FN"] = FN
        F1df_counts[sample,"FP"] = FP
        F1df_counts[sample,"TP"] = TP
        F1df_counts[sample,"TAR"] = TP / (TP + FP)
        F1df_counts[sample,"TDR"] = TP / (TP + FN)

        num = F1df_counts[sample,"TAR"]*F1df_counts[sample,"TDR"]
        den = F1df_counts[sample,"TAR"]+F1df_counts[sample,"TDR"] 
        F1df_counts[sample,"F1"] = 2*(num/den)
        F1df_counts[sample,"Subject.ID"] = OTUdf_in[sample,"Subject.ID"]
        F1df_counts[sample,"bmark"] = bmark_in
    }
    return(F1df_counts)
}

In [14]:
#F1_in=F1df_counts
bench_means <- function(F1_in){
    df = with(F1_in, tapply(X = FN,       
                    INDEX = Subject.ID,   
                    FUN = mean,     
                    na.rm = TRUE))
    return(df)
}

In [15]:
plot_violin <- function(df_in,x_lab,y_ax,y_lab,col_in){
    p = ggplot(df_in, aes(x=get(x_lab), y=get(y_ax), color=get(col_in))) + 
        geom_violin(trim=FALSE) + 
        labs(color = col_in) +
        theme(axis.text.x = element_text(angle = 90, hjust = 1),
              axis.title.x = element_blank()) + 
        ylab(y_lab) +
        stat_summary(fun.data=mean_sdl, color="red")
    return (p)
}

In [31]:
plot_violin_in <- function(F1_in,x_list,y_list,id_in,bench_key,fname){
    
    p_1 = plot_violin(F1_in,id_in,x_list[1],y_list[1],id_in)
    p_2 = plot_violin(F1_in,id_in,x_list[2],y_list[2],id_in)
    p_3 = plot_violin(F1_in,id_in,x_list[3],y_list[3],id_in)

    #Create count violin plots

    fig = ggarrange(p_1, p_2, p_3, 
              labels = c("A", "B", "C"),
              title = bench_key,
              common.legend = TRUE,
              ncol = 2, nrow = 2)
    annotate_figure(fig, #https://rpkgs.datanovia.com/ggpubr/reference/annotate_figure.html
                   top = text_grob(bench_key, color = "blue", face = "bold", size = 14)
    )
    
    ggsave(paste(out_dir,"/img/violinplot_",bench_key,"_",fname,".jpg",sep=""))
}

In [30]:
#Create a merged df for all variables
mergedf = data.frame()

In [32]:
for (variable in unique(benchmarkdf$x)){
    print("\n****************************************************************************")
    print(paste("Analysis for ",variable))
    
    #Create the phyloseq object
    filename = paste(out_dir,'taxonomic_classification/',variable,sep="")
    phy_obj = phylo_create(filename)  
    
    #Create glomed, subsetted obj
    seq_obj = phy_seq(phy_obj)
    remove(phy_obj)
    
    #Create OTUdf
    OTUdf = otu_extract(seq_obj)
    
    #Create TAXdf
    TAXdf = tax_extract(seq_obj)
    
    #Clean df's
    OTUdf_clean = otu_tax_clean(OTUdf,TAXdf)
    
    #Create TP,FP,FN df
    tax_ref = as.character(benchmarkdf[variable,"tax.ref"])
    if(tax_ref=='silva'){
        F1df = bench_compare(OTUdf_clean, CONTdf_s)
    } else{
        F1df = bench_compare(OTUdf_clean, CONTdf_g)
    }
    
    #Create TP,FP,FN counts    
    if(benchmarkdf[variable,"group.type"]=="dada" | benchmarkdf[variable,"group.type"]=="dn"){
        bench_key = paste(benchmarkdf[variable,"group.type"],benchmarkdf[variable,"tax.type"],
                         benchmarkdf[variable,"tax.ref"],sep="_")
    } else{
        bench_key = paste(benchmarkdf[variable,"group.type"],benchmarkdf[variable,"group.ref"],
                         benchmarkdf[variable,"tax.type"],benchmarkdf[variable,"tax.ref"],sep="_")
    }
    
    F1df_counts = bench_counts(OTUdf_clean,F1df,bench_key)
    #write.csv(F1df_counts,paste(out_dir,tax_ref,"test.csv",sep=""))
    
    #Create violin plots for each control
    x_list = c("FN","TP","FP")
    y_labs = c("False negative","True positive", "False positive")
    plot_violin_in(F1df_counts,x_list,y_labs,"Subject.ID",bench_key,"counts")
    
    #Create violin plots for each control
    x_list = c("TAR","TDR","F1")
    y_labs = c("Taxonmic Accuracy Rate","Taxonmic Detection Rate", "F-Measure")
    plot_violin_in(F1df_counts,x_list,y_labs,"Subject.ID",bench_key,"summary")

    #Merge dfs
    temp = F1df_counts
    temp$sample_name = rownames(temp)
    mergedf = rbind(mergedf,temp)
}

[1] "\n****************************************************************************"
[1] "Analysis for  scikit_dada_SILVA.qza"
phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 1818 taxa and 351 samples ]
sample_data() Sample Data:       [ 351 samples by 24 sample variables ]
tax_table()   Taxonomy Table:    [ 1818 taxa by 7 taxonomic ranks ]
[1] "number of taxa before: 1818"
[1] "number of taxa after glom:  271"


Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image



[1] "\n****************************************************************************"
[1] "Analysis for  scikit_dada_gg.qza"
phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 1818 taxa and 351 samples ]
sample_data() Sample Data:       [ 351 samples by 24 sample variables ]
tax_table()   Taxonomy Table:    [ 1818 taxa by 7 taxonomic ranks ]
[1] "number of taxa before: 1818"
[1] "number of taxa after glom:  174"


Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image



In [35]:
mergedf

,Subject.ID,check,FN,FP,TP,TAR,TDR,F1,bmark,sample_name
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
SC284730,MSA1000,10,2,3,8,0.7272727,0.8000000,0.7619048,dada_scikit_silva,SC284730
SC284731,MSA1001,10,2,4,8,0.6666667,0.8000000,0.7272727,dada_scikit_silva,SC284731
SC284732,MSA1002,18,3,4,15,0.7894737,0.8333333,0.8108108,dada_scikit_silva,SC284732
SC284733,MSA1003,18,4,6,14,0.7000000,0.7777778,0.7368421,dada_scikit_silva,SC284733
SC284734,D6305,8,1,1,7,0.8750000,0.8750000,0.8750000,dada_scikit_silva,SC284734
SC284735,D6306,8,1,16,7,0.3043478,0.8750000,0.4516129,dada_scikit_silva,SC284735
SC555338,D6311,8,4,0,4,1.0000000,0.5000000,0.6666667,dada_scikit_silva,SC555338
SC555339,MSA1001,10,2,2,8,0.8000000,0.8000000,0.8000000,dada_scikit_silva,SC555339
SC555340,D6305,8,1,1,7,0.8750000,0.8750000,0.8750000,dada_scikit_silva,SC555340


In [36]:
#Create violin plots for each control
x_list = c("TAR","TDR","F1")
y_labs = c("Taxonmic Accuracy Rate","Taxonmic Detection Rate", "F-Measure")
plot_violin_in(mergedf,x_list,y_labs,"bmark","All Grouping and Assignment Variables","summary")

Saving 6.67 x 6.67 in image

